In [5]:
import numpy as np
np.random.seed(11)
import tensorflow as tf
tf.random.set_seed(11)

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.utils import np_utils, load_img, img_to_array

In [7]:
from sklearn.datasets import load_files
from tqdm import tqdm

## Load images

In [8]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 102)
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('./Data/Datasplit/train')
valid_files, valid_targets = load_dataset('./Data/Datasplit/valid')
test_files, test_targets = load_dataset('./Data/Datasplit/test')

## Convert images to float

In [9]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

## Build a CNN model from scratch

In [10]:
train_tensors = paths_to_tensor(train_files)
valid_tensors = paths_to_tensor(valid_files)
test_tensors = paths_to_tensor(test_files)

100%|██████████████████████████████████████| 6149/6149 [00:58<00:00, 104.80it/s]


In [11]:
cnn = Sequential()
cnn.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(224, 224, 3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Dropout(0.3))
cnn.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(102, activation='softmax'))
cnn.summary()

2023-02-07 15:49:51.974848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        1

In [12]:
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets), 
        epochs=10, batch_size=20, verbose=0)

In [20]:
cnn.evaluate(train_tensors, train_targets)
cnn.evaluate(valid_tensors, valid_targets)
cnn.evaluate(test_tensors, test_targets)

193/193 [==============================] - 65s 330ms/step - loss: 3.7849 - accuracy: 0.1278


[3.784881830215454, 0.12782566249370575]

In [15]:
# get prediction
flower_preds = [np.argmax(cnn.predict(np.expand_dims(tensor, axis=0), verbose=0)) for tensor in test_tensors]

In [18]:
# another way to get accuracy
test_accuracy = 100*np.sum(np.array(flower_preds)==np.argmax(test_targets, axis=1))/len(flower_preds)
print(f'Test accuracy: {test_accuracy:.2f}%')

Test accuracy: 12.78%
